# __......................................................بسم الله الرحمن الرحيم......................................................__

# __................................................Image segmentation................................................__

## __Notebook Overview:__

__we implement image segmentation using U-net to build a segmentation model.__

In this notebook:

  • we will apply augmentation on dataset.

  • we are using data set for people dancing from Tiktok short videos.

  • the mode will be able to segment pictures or videos that include humans moving in it.

 __• Tasks:__

[1- Importing, Exploring and Augmenting Data.](#1)
<a id='5'></a>

[2- Bulding the Model.](#4)
<a id='8'></a>

[3- Evaluating the Model.](#5)
<a id='9'></a>

## __Notabook Code:__

<a id='1'></a>
### [__1- Importing, Exploring and Augmenting Data:__](#5)

__Importing Data:__

In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download tapakah68/segmentation-full-body-tiktok-dancing-dataset

100% 1.81G/1.82G [00:25<00:00, 64.5MB/s]
100% 1.82G/1.82G [00:25<00:00, 76.6MB/s]


In [ ]:
! unzip segmentation-full-body-tiktok-dancing-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/images/137_00390.png  
  inflating: segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/images/137_00420.png  
  inflating: segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/images/139_00120.png  
  inflating: segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/images/139_00210.png  
  inflating: segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/images/139_00240.png  
  inflating: segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/images/141_00090.png  
  inflating: segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/images/141_00120.png  
  inflating: segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/images/141_00150.png  
  inflating: segmenta

__Importing Librarys:__

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from albumentations import HorizontalFlip, VerticalFlip, Rotate

In [ ]:
BASE_DIR = '/content/'
WORKING_DIR = '/content/'

__Augmentation Functions:__

In [ ]:
def hFlip(image, mask):
  aug = HorizontalFlip(p=1.0)
  augmented = aug(image=image, mask=mask)
  return augmented

def vFlip(image, mask):
  aug = VerticalFlip(p=1.0)
  augmented = aug(image=image, mask=mask)
  return augmented

def rotate(image, mask):
  aug = Rotate(limit=45, p=1.0)
  augmented = aug(image=image, mask=mask)
  return augmented


def augment_data(images, masks, save_path, augment=True, H=512, W=512):
    """ Performing data augmentation. """

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting the dir name and image name """
        dir_name = x.split("/")[-2]
        name = dir_name + "_" + x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        if augment == True:

            augmented = hFlip(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            augmented = vFlip(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            augmented = rotate(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

        else:
            X = [x]
            #print(len(X))
            Y =[y]


        #split it
        idx = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))
            m = m/255.0
            m = (m > 0.5) * 255

            if len(X) == 1:  # each loop it will take one image
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name  = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{idx}.jpg"
                tmp_mask_name  = f"{name}_{idx}.jpg"

            image_path = os.path.join(save_path, "image/", tmp_image_name)
            mask_path  = os.path.join(save_path, "mask/", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1

__Loadding Function:__

In [ ]:
def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path, split=0.2):
    """ Load the images and masks """
    images = sorted(glob(f"{path}/images/*.png"))
    masks = sorted(glob(f"{path}/masks/*.png"))

    """ Split the data """
    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y)

In [ ]:
dataset_path = BASE_DIR + 'segmentation_full_body_tik_tok_2615_img/segmentation_full_body_tik_tok_2615_img/'
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path, split=0.2)

In [ ]:
print("Train: ", len(train_x))
print("Valid: ", len(valid_x))

Train:  2092
Valid:  523


In [ ]:
create_dir(BASE_DIR + "new_data/train/image/")
create_dir(BASE_DIR + "new_data/train/mask/")
create_dir(BASE_DIR + "new_data/valid/image/")
create_dir(BASE_DIR + "new_data/valid/mask/")


In [ ]:
augment_data(train_x, train_y, "new_data/train/", augment=True)
augment_data(valid_x, valid_y, "new_data/valid/", augment=False)

100%|██████████| 523/523 [00:26<00:00, 19.49it/s]


<a id='1'></a>
### [__2- Bulding the Model.__](#5)

In [ ]:

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

H = 512
W = 512

def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("/content/files")

""" Hyperparameters """
batch_size = 2
lr = 1e-4
num_epochs = 5
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "data.csv")

""" Dataset """
train_path = BASE_DIR +  "new_data/train/"
valid_path = BASE_DIR +  "new_data/valid/"

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

Train: 8368 - 8368
Valid: 523 - 523


In [ ]:
""" Model """
model = build_unet((H, W, 3))
metrics = [dice_coef, iou, Recall(), Precision()]
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

In [ ]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks,
    shuffle=False
)

Epoch 1/5
1976/4184 [=============>................] - ETA: 18:20 - loss: 0.3746 - dice_coef: 0.6254 - iou: 0.4745 - recall: 0.7365 - precision: 0.5725

In [ ]:


model.save(BASE_DIR + 'model.h5')

In [ ]:
from google.colab import files

files.download(BASE_DIR + 'model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<a id='1'></a>
### [__3- Evaluating the Model.__](#5)

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

H = 512
W = 512

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def save_results(image, mask, y_pred, save_image_path):
    ## i - m - y
    line = np.ones((H, 10, 3)) * 128

    """ Mask """
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)

    """ Predicted Mask """
    y_pred = np.expand_dims(y_pred, axis=-1)    ## (512, 512, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)  ## (512, 512, 3)
    y_pred = y_pred * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)



In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir(BASE_DIR + "results")

""" Loading model """
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
    model = tf.keras.models.load_model(BASE_DIR + "files/model.h5")

""" Load the dataset """
test_x = sorted(glob(os.path.join("new_data", "valid", "image", "*")))
test_y = sorted(glob(os.path.join("new_data", "valid", "mask", "*")))
print(f"Test: {len(test_x)} - {len(test_y)}")

""" Evaluation and Prediction """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extract the name """
    name = x.split("/")[-1].split(".")[0]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    x = image/255.0
    x = np.expand_dims(x, axis=0)

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    y = mask/255.0
    y = y > 0.5
    y = y.astype(np.int32)

    """ Prediction """
    y_pred = model.predict(x)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = BASE_DIR + f"results/{name}.png"
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

""" Metrics values """
score = [s[1:]for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")

df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv(BASE_DIR + "files/score.csv")

Test: 523 - 523


  0%|          | 0/523 [00:00<?, ?it/s]

1/1 [==============================] - 1s 616ms/step


  0%|          | 1/523 [00:01<10:31,  1.21s/it]

1/1 [==============================] - 0s 31ms/step


...

1/1 [==============================] - 0s 32ms/step


100%|█████████▉| 522/523 [03:52<00:00,  1.71it/s]

1/1 [==============================] - 0s 29ms/step


100%|██████████| 523/523 [03:53<00:00,  2.24it/s]

Accuracy: 0.97999
F1: 0.89114
Jaccard: 0.82085
Recall: 0.88826
Precision: 0.91310


# __...........................................................تمت بحمد الله...........................................................__